# Objectives

* Notebook to load 3D T/S
* create xgcm.Grid

In [1]:
import numpy as np
import xarray as xr
import cf_xarray
from xgcm import Grid, generate_grid_ds
import gsw_xarray as gsw

In [2]:
ds = xr.open_mfdataset("../data/woa23*.nc", decode_times=False).squeeze("time", drop=True)[["t_an","s_an"]]
# We only keep the objectively analyzed mean fields
ds = ds.assign_coords({"lon":ds.lon%360}).sortby("lon")
ds.lon.attrs["standard_name"] = "longitude"

In [3]:
ds

<xarray.Dataset> Size: 53MB
Dimensions:  (depth: 102, lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float32 720B -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * depth    (depth) float32 408B 0.0 5.0 10.0 15.0 ... 5.3e+03 5.4e+03 5.5e+03
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    t_an     (depth, lat, lon) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    s_an     (depth, lat, lon) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
Attributes: (12/45)
    Conventions:                     CF-1.6
    title:                           World Ocean Atlas 2023 : sea_water_pract...
    summary:                         Climatological mean salinity for the glo...
    references:                      Reagan, J.R., D. Seidov, Z. Wang, D. Duk...
    institution:                     NOAA National Centers for Environmental ...
    comment:                         Global Climatology as part of the World ...
    ...                              ...
    ncei_template_version:           NCEI_NetCDF_Grid_Template_v1.0
    license:                         These data are openly available to the p...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    metadata_link:                   https://www.ncei.noaa.gov/products/world...
    date_created:                    2024-01-28 
    date_modified:                   2024-01-28

In [4]:
ds["p"] = gsw.p_from_z(z=-ds["depth"], lat=ds.lat)

In [5]:
# add CT and SA
ds["SA"] = ds.gsw.SA_from_SP()

In [6]:
ds["CT"] = ds.gsw.CT_from_t()

## Create xgcm.Grid

1. Create new dims: lon/lat/depth to x_c, y_c, z_c
1. Create new x_f, y_f, z_f dimensions that correspond to cell faces and add proper attributes
2. Add metrics
3. Create final grid

In [7]:
ds = generate_grid_ds(
    ds,
    {"X":ds.cf["longitude"].name, "Y":ds.cf["latitude"].name, "Z":ds.cf["depth"].name},
    position={'X':('center','right'),'Y':('center','right'),'Z':('center','right')}
)#.rename({f"{x}_c_right":f"{x}_f" for x in "xyz"})
ds

<xarray.Dataset> Size: 159MB
Dimensions:      (depth: 102, lat: 180, lon: 360, lon_right: 360,
                  lat_right: 180, depth_right: 102)
Coordinates:
  * lat          (lat) float32 720B -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth        (depth) float32 408B 0.0 5.0 10.0 ... 5.3e+03 5.4e+03 5.5e+03
  * lon          (lon) float32 1kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * lon_right    (lon_right) float32 1kB 1.0 2.0 3.0 4.0 ... 358.0 359.0 360.0
  * lat_right    (lat_right) float32 720B -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * depth_right  (depth_right) float32 408B 2.5 7.5 12.5 ... 5.45e+03 5.55e+03
Data variables:
    t_an         (depth, lat, lon) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    s_an         (depth, lat, lon) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    p            (depth, lat) float64 147kB 0.0 0.0 0.0 ... 5.632e+03 5.632e+03
    SA           (depth, lat, lon) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    CT           (depth, lat, lon) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
Attributes: (12/45)
    Conventions:                     CF-1.6
    title:                           World Ocean Atlas 2023 : sea_water_pract...
    summary:                         Climatological mean salinity for the glo...
    references:                      Reagan, J.R., D. Seidov, Z. Wang, D. Duk...
    institution:                     NOAA National Centers for Environmental ...
    comment:                         Global Climatology as part of the World ...
    ...                              ...
    ncei_template_version:           NCEI_NetCDF_Grid_Template_v1.0
    license:                         These data are openly available to the p...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    metadata_link:                   https://www.ncei.noaa.gov/products/world...
    date_created:                    2024-01-28 
    date_modified:                   2024-01-28

In [8]:
for xyz, lld in zip(["x","y","z"],["longitude","latitude","depth"]):
    ds[f"{xyz}_c"] = (ds.cf[lld].name, np.arange(ds.cf[lld].shape[0]))
    ds = ds.swap_dims({ds.cf[lld].name:f"{xyz}_c"})
    ds[f"{xyz}_f"] = (ds.cf[lld].name+"_right", np.arange(ds.cf[lld].shape[0]))
    ds = ds.swap_dims({ds.cf[lld].name+"_right":f"{xyz}_f"})
    ds[f"{xyz}_c"].attrs["axis"] = ds[ds.cf[lld].name].attrs.pop("axis")
    ds[f"{xyz}_f"].attrs["axis"] = ds[ds.cf[lld].name+"_right"].attrs.pop("axis")
    ds[f"{xyz}_f"].attrs["c_grid_axis_shift"] = ds[ds.cf[lld].name+"_right"].attrs.pop("c_grid_axis_shift")
    ds[ds.cf[lld].name+"_right"].attrs["standard_name"] = lld
ds

<xarray.Dataset> Size: 159MB
Dimensions:      (z_c: 102, y_c: 180, x_c: 360, x_f: 360, y_f: 180, z_f: 102)
Coordinates:
    lat          (y_c) float32 720B -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
    depth        (z_c) float32 408B 0.0 5.0 10.0 ... 5.3e+03 5.4e+03 5.5e+03
    lon          (x_c) float32 1kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
    lon_right    (x_f) float32 1kB 1.0 2.0 3.0 4.0 ... 357.0 358.0 359.0 360.0
    lat_right    (y_f) float32 720B -89.0 -88.0 -87.0 -86.0 ... 88.0 89.0 90.0
    depth_right  (z_f) float32 408B 2.5 7.5 12.5 ... 5.35e+03 5.45e+03 5.55e+03
  * x_c          (x_c) int64 3kB 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
  * x_f          (x_f) int64 3kB 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
  * y_c          (y_c) int64 1kB 0 1 2 3 4 5 6 7 ... 173 174 175 176 177 178 179
  * y_f          (y_f) int64 1kB 0 1 2 3 4 5 6 7 ... 173 174 175 176 177 178 179
  * z_c          (z_c) int64 816B 0 1 2 3 4 5 6 7 ... 94 95 96 97 98 99 100 101
  * z_f          (z_f) int64 816B 0 1 2 3 4 5 6 7 ... 94 95 96 97 98 99 100 101
Data variables:
    t_an         (z_c, y_c, x_c) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    s_an         (z_c, y_c, x_c) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    p            (z_c, y_c) float64 147kB 0.0 0.0 0.0 ... 5.632e+03 5.632e+03
    SA           (z_c, y_c, x_c) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    CT           (z_c, y_c, x_c) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
Attributes: (12/45)
    Conventions:                     CF-1.6
    title:                           World Ocean Atlas 2023 : sea_water_pract...
    summary:                         Climatological mean salinity for the glo...
    references:                      Reagan, J.R., D. Seidov, Z. Wang, D. Duk...
    institution:                     NOAA National Centers for Environmental ...
    comment:                         Global Climatology as part of the World ...
    ...                              ...
    ncei_template_version:           NCEI_NetCDF_Grid_Template_v1.0
    license:                         These data are openly available to the p...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    metadata_link:                   https://www.ncei.noaa.gov/products/world...
    date_created:                    2024-01-28 
    date_modified:                   2024-01-28

In [9]:
grid = Grid(ds, periodic=["X"])
grid

<xgcm.Grid>
Y Axis (not periodic, boundary=None):
  * center   y_c --> right
  * right    y_f --> center
X Axis (periodic, boundary=None):
  * center   x_c --> right
  * right    x_f --> center
Z Axis (not periodic, boundary=None):
  * center   z_c --> right
  * right    z_f --> center

In [10]:
dlong = 1
dlonc = 1

dlatg =1
dlatc = 1

In [11]:
def dll_dist(dlon, dlat, lon, lat):
        """Converts lat/lon differentials into distances in meters

        PARAMETERS
        ----------
        dlon : xarray.DataArray longitude differentials
        dlat : xarray.DataArray latitude differentials
        lon  : xarray.DataArray longitude values
        lat  : xarray.DataArray latitude values

        RETURNS
        -------
        dx  : xarray.DataArray distance inferred from dlon
        dy  : xarray.DataArray distance inferred from dlat
        """

        distance_1deg_equator = 111000.0
        dx = dlon * np.cos(np.deg2rad(lat)) * distance_1deg_equator
        dy = ((lon * 0) + 1) * dlat * distance_1deg_equator
        return dx, dy

ds.coords['e1tu'], e2 = dll_dist(dlonc, dlatc, ds.lon, ds.lat)
# e1tu is at u and t points => function of y_c
ds.coords['e2'] = np.nanmean(e2)
ds.coords['e1vf'], e2 = dll_dist(dlonc, dlatc, ds.lon, ds.lat_right)
# e1vf is at v and f points => function of y_f

In [12]:
ds.coords["e3w"] = grid.diff(
    ds["depth"],
    "Z",
    boundary="fill",
    fill_value=2*ds["depth"].values[-1] - ds["depth"].values[-2]
)

/home/romain/.cache/pypoetry/virtualenvs/neutral-density-BeRXjZM7-py3.10/lib/python3.10/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


In [13]:
ds.coords["e3t"] = grid.diff(
    ds["depth_right"],
    "Z",
    boundary="fill",
    fill_value=-ds["depth_right"].values[0]
)

In [14]:
ds

<xarray.Dataset> Size: 159MB
Dimensions:      (z_c: 102, y_c: 180, x_c: 360, x_f: 360, y_f: 180, z_f: 102)
Coordinates: (12/17)
    lat          (y_c) float32 720B -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
    depth        (z_c) float32 408B 0.0 5.0 10.0 ... 5.3e+03 5.4e+03 5.5e+03
    lon          (x_c) float32 1kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
    lon_right    (x_f) float32 1kB 1.0 2.0 3.0 4.0 ... 357.0 358.0 359.0 360.0
    lat_right    (y_f) float32 720B -89.0 -88.0 -87.0 -86.0 ... 88.0 89.0 90.0
    depth_right  (z_f) float32 408B 2.5 7.5 12.5 ... 5.35e+03 5.45e+03 5.55e+03
    ...           ...
  * z_f          (z_f) int64 816B 0 1 2 3 4 5 6 7 ... 94 95 96 97 98 99 100 101
    e1tu         (y_c) float32 720B 968.6 2.906e+03 ... 2.906e+03 968.6
    e2           float32 4B 1.11e+05
    e1vf         (y_f) float32 720B 1.937e+03 3.874e+03 ... 1.937e+03 -0.004852
    e3w          (z_f) float32 408B 5.0 5.0 5.0 5.0 ... 100.0 100.0 100.0 100.0
    e3t          (z_c) float32 408B 5.0 5.0 5.0 5.0 ... 100.0 100.0 100.0 100.0
Data variables:
    t_an         (z_c, y_c, x_c) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    s_an         (z_c, y_c, x_c) float32 26MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    p            (z_c, y_c) float64 147kB 0.0 0.0 0.0 ... 5.632e+03 5.632e+03
    SA           (z_c, y_c, x_c) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
    CT           (z_c, y_c, x_c) float64 53MB dask.array<chunksize=(51, 90, 180), meta=np.ndarray>
Attributes: (12/45)
    Conventions:                     CF-1.6
    title:                           World Ocean Atlas 2023 : sea_water_pract...
    summary:                         Climatological mean salinity for the glo...
    references:                      Reagan, J.R., D. Seidov, Z. Wang, D. Duk...
    institution:                     NOAA National Centers for Environmental ...
    comment:                         Global Climatology as part of the World ...
    ...                              ...
    ncei_template_version:           NCEI_NetCDF_Grid_Template_v1.0
    license:                         These data are openly available to the p...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    metadata_link:                   https://www.ncei.noaa.gov/products/world...
    date_created:                    2024-01-28 
    date_modified:                   2024-01-28

In [15]:
ds.to_netcdf("../data/processed_woa.nc")

In [16]:
grid = Grid(ds, periodic=["X"], metrics={"X":["e1tu","e1vf"], "Y":["e2"], "Z":["e3t","e3w"]})
grid

<xgcm.Grid>
Y Axis (not periodic, boundary=None):
  * center   y_c --> right
  * right    y_f --> center
X Axis (periodic, boundary=None):
  * center   x_c --> right
  * right    x_f --> center
Z Axis (not periodic, boundary=None):
  * center   z_c --> right
  * right    z_f --> center